We'll use https://cassiopeia.readthedocs.io/en/latest/ to download data.

In [1]:
import pandas as pd
aa = pd.read_pickle('data2.pkl')
print(len(aa))

aa = [a for a in aa if a!=[]]  # remove empty history
df = pd.DataFrame(aa)
df = df.loc[df[20].isin([True, False])]
df.shape

110001


(13590, 21)

In [2]:
df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,레오나,리 신,모데카이저,베인,이렐리아,164.0,236.0,266.0,117.0,555.0,...,카이사,야스오,탈론,럭스,555.0,69.0,38.0,3.0,55.0,False
1,이블린,쉔,판테온,갈리오,이렐리아,67.0,69.0,555.0,24.0,64.0,...,니달리,트리스타나,나미,카서스,238.0,12.0,24.0,157.0,555.0,True
2,이즈리얼,갈리오,아트록스,그라가스,파이크,236.0,39.0,6.0,64.0,518.0,...,티모,케인,베인,쓰레쉬,69.0,112.0,236.0,157.0,518.0,False
3,렝가,케이틀린,직스,베이가,카직스,112.0,7.0,236.0,84.0,122.0,...,판테온,럭스,스웨인,갈리오,17.0,266.0,157.0,64.0,518.0,True
4,우르곳,카직스,블라디미르,바루스,쓰레쉬,81.0,122.0,53.0,518.0,157.0,...,카이사,리 신,파이크,아리,3.0,39.0,203.0,69.0,266.0,False
5,아트록스,니달리,케이틀린,쓰레쉬,아리,420.0,64.0,518.0,53.0,121.0,...,마스터 이,요릭,진,다이애나,12.0,7.0,69.0,3.0,518.0,False
6,제라스,이즈리얼,럭스,레넥톤,람머스,38.0,7.0,3.0,145.0,39.0,...,블리츠크랭크,케인,우르곳,루시안,518.0,122.0,157.0,69.0,555.0,True
7,아리,카밀,다리우스,베인,쓰레쉬,236.0,69.0,112.0,518.0,64.0,...,판테온,모르가나,트위치,야스오,38.0,84.0,266.0,53.0,63.0,True
8,케이틀린,카밀,아칼리,벨코즈,아트록스,69.0,236.0,64.0,5.0,17.0,...,일라오이,미스 포츈,카타리나,애니,238.0,555.0,518.0,157.0,11.0,False
9,사이온,야스오,브랜드,리 신,마오카이,89.0,122.0,99.0,39.0,7.0,...,아무무,잭스,제라스,카이사,420.0,53.0,236.0,38.0,238.0,False


In [3]:
import getpass
import random

key = getpass.getpass("Enter Riot API key:")

Enter Riot API key:········


In [4]:
from cassiopeia import Champion, Champions, Summoner

import cassiopeia as cass

cass.set_riot_api_key(key)  # This overrides the value set in your configuration/settings.
cass.set_default_region("KR")

In [5]:
import random

import cassiopeia as cass

cass.set_riot_api_key(key)  # This overrides the value set in your configuration/settings.
cass.set_default_region("KR")

summoner = cass.get_summoner(name="dskjbfksaf")
print("{name} is a level {level} summoner on the {region} server.".format(name=summoner.name,
                                                                          level=summoner.level,
                                                                          region=summoner.region))

champions = cass.get_champions()
random_champion = random.choice(champions)
print("He enjoys playing champions such as {name}.".format(name=random_champion.name))

challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives)
best_na = challenger_league[0].summoner
print("He's not as good as {name} at League, but probably a better python programmer!".format(name=best_na.name))

Making call: https://kr.api.riotgames.com/lol/summoner/v3/summoners/by-name/dskjbfksaf
dskjbfksaf is a level 59 summoner on the Region.korea server.
Making call: https://ddragon.leagueoflegends.com/realms/kr.json
Making call: https://ddragon.leagueoflegends.com/cdn/9.1.1/data/ko_KR/championFull.json
He enjoys playing champions such as 자크.
Making call: https://kr.api.riotgames.com/lol/league/v3/challengerleagues/by-queue/RANKED_SOLO_5x5
He's not as good as 나유준 at League, but probably a better python programmer!


In [8]:
mm = cass.get_match(3475605500)

In [20]:
pp = mm.participants[0]

In [ ]:
pp.

In [23]:
for i in range(10) :
    print(mm.participants[i].summoner.name, mm.participants[i].id)

감자캐리힘 1
끌꺼라고 믿지마 2
간장토마토 3
Tahm Kench most1 4
가자박태환 5
메가 거북왕 6
대일고 7
단두대가 단 두대 8
에그코 9
반억왕다리우스 10


In [22]:
for i in range(5) :
    print(mm.blue_team.participants[i].summoner.name)

감자캐리힘
끌꺼라고 믿지마
간장토마토
Tahm Kench most1
가자박태환


In [53]:
import cassiopeia as cass

summoner = cass.get_summoner(name="벤젠덴겐")
match_hist = cass.get_match_history(summoner)

In [128]:
tmp = match_hist[10].blue_team.bans[2].id

In [129]:
tmp

64

In [111]:
RANKED_SOLO_5x5
RANKED_FLEX_SR

'RANKED_FLEX_SR'

In [108]:
type(tmp.ranked_solo_fives)

<enum 'Queue'>

In [124]:
for i in range(5) : 
    print(match_hist[0].red_team.participants[i].summoner.name)
    print(match_hist[0].red_team.participants[i].champion.id)

앙띠미니
127
벤젠덴겐
236
지원되지않는이름
15
공간의 파수꾼
134
Les jupes
104


In [132]:
from cassiopeia import Champion, Champions

champions = Champions(region="KR")
for champion in champions:
    print(champion.name, champion.id)

아트록스 266
아리 103
아칼리 84
알리스타 12
아무무 32
애니비아 34
애니 1
애쉬 22
아우렐리온 솔 136
아지르 268
바드 432
블리츠크랭크 53
브랜드 63
브라움 201
케이틀린 51
카밀 164
카시오페아 69
초가스 31
코르키 42
다리우스 122
다이애나 131
드레이븐 119
문도 박사 36
에코 245
엘리스 60
이블린 28
이즈리얼 81
피들스틱 9
피오라 114
피즈 105
갈리오 3
갱플랭크 41
가렌 86
나르 150
그라가스 79
그레이브즈 104
헤카림 120
하이머딩거 74
일라오이 420
이렐리아 39
아이번 427
잔나 40
자르반 4세 59
잭스 24
제이스 126
진 202
징크스 222
카이사 145
칼리스타 429
카르마 43
카서스 30
카사딘 38
카타리나 55
케일 10
케인 141
케넨 85
카직스 121
킨드레드 203
클레드 240
코그모 96
르블랑 7
리 신 64
레오나 89
리산드라 127
루시안 236
룰루 117
럭스 99
말파이트 54
말자하 90
마오카이 57
마스터 이 11
미스 포츈 21
오공 62
모데카이저 82
모르가나 25
나미 267
나서스 75
노틸러스 111
니코 518
니달리 76
녹턴 56
누누와 윌럼프 20
올라프 2
오리아나 61
오른 516
판테온 80
뽀삐 78
파이크 555
퀸 133
라칸 497
람머스 33
렉사이 421
레넥톤 58
렝가 107
리븐 92
럼블 68
라이즈 13
세주아니 113
샤코 35
쉔 98
쉬바나 102
신지드 27
사이온 14
시비르 15
스카너 72
소나 37
소라카 16
스웨인 50
신드라 134
탐 켄치 223
탈리야 163
탈론 91
타릭 44
티모 17
쓰레쉬 412
트리스타나 18
트런들 48
트린다미어 23
트위스티드 페이트 4
트위치 29
우디르 77
우르곳 6
바루스 110
베인 67
베이가 45
벨코즈 161
바이 254
빅토르 112
블라디미르 8
볼리베어 106
워윅 19
자야 498
제라스 101
신

In [17]:
match_hist[0].blue_team.win

True

In [9]:
match_hist[0].id

3449425180

In [10]:
import cassiopeia as cass

match_hist[0].map.name

Making call: https://ddragon.leagueoflegends.com/cdn/8.24.1/data/ko_KR/map.json


'소환사의 협곡'

In [11]:
match_hist = cass.get_match_history(summoner)

In [12]:
for i in range(5) : 
    if match_hist[0].teams[0].win : 
        print('blue team : ', match_hist[0].teams[0].participants[i].summoner.name, 'win')
    else :
        print('blue team : ', match_hist[0].teams[0].participants[i].summoner.name, 'loose')
for i in range(5) :
    if match_hist[0].teams[1].win : 
        print('red team : ', match_hist[0].teams[1].participants[i].summoner.name, 'win')
    else :
        print('red team : ', match_hist[0].teams[1].participants[i].summoner.name, 'loose')

Making call: https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/5500568?beginIndex=0&endIndex=100
blue team :  소중한한명 win
blue team :  벤젠덴겐 win
blue team :  브라질리언했음 win
blue team :  머스켓 win
blue team :  피들장인 최빨대 win
red team :  탐스런콧구멍 loose
red team :  까치래도차케 loose
red team :  horosha loose
red team :  시로조아 loose
red team :  Ezreal Lover loose


In [13]:
match_hist[0].participants[summoner.name].champion.id

6

In [14]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter


def print_newest_match(name: str, region: str):

    # Notice how this function never makes a call to the summoner endpoint because we provide all the needed data!

    summoner = Summoner(name=name)

    # A MatchHistory is a lazy list, meaning it's elements only get loaded as-needed.

    # match_history = cass.get_match_history(summoner=summoner, seasons={Season.season_8}, queues={Queue.ranked_solo_fives})
    match_history = cass.get_match_history(summoner)
    match_history(seasons={Season.season_8}, queues={Queue.ranked_solo_fives})

    # Load the entire match history by iterating over all its elements so that we know how long it is.
    # Unfortunately since we are iterating over the match history and accessing the summoner's champion for each match,
    # we need to know what version of the static data the champion should have. To avoid pulling many different
    # static data versions, we will instead create a {champion_id -> champion_name} mapping and just access the champion's
    # ID from the match data (which it provides directly).
    champion_id_to_name_mapping = {champion.id: champion.name for champion in cass.get_champions(region=region)}
    played_champions = Counter()
    for match in match_history:
        champion_id = match.participants[summoner.name].champion.id
        champion_name = champion_id_to_name_mapping[champion_id]
        played_champions[champion_name] += 1
    print("Length of match history:", len(match_history))

    # Print the aggregated champion results
    print("Top 10 champions {} played:".format(summoner.name))
    for champion_name, count in played_champions.most_common(10):
        print(champion_name, count)
    print()

print_newest_match(name='벤젠덴겐', region='KR')


Making call: https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/209691368?beginIndex=0&endIndex=100
Making call: https://ddragon.leagueoflegends.com/cdn/8.24.1/data/ko_KR/item.json
Making call: https://ddragon.leagueoflegends.com/cdn/8.24.1/data/ko_KR/runesReforged.json


ValueError: '영감' is not a valid RunePath

In [ ]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter


def print_newest_match(name: str, region: str):

    # Notice how this function never makes a call to the summoner endpoint because we provide all the needed data!

    summoner = Summoner(name=name, region=region)

    # A MatchHistory is a lazy list, meaning it's elements only get loaded as-needed.

    # match_history = cass.get_match_history(summoner=summoner, seasons={Season.season_8}, queues={Queue.ranked_solo_fives})
    match_history = summoner.match_history
    match_history(seasons={Season.season_8}, queues={Queue.ranked_solo_fives})

    # Load the entire match history by iterating over all its elements so that we know how long it is.
    # Unfortunately since we are iterating over the match history and accessing the summoner's champion for each match,
    # we need to know what version of the static data the champion should have. To avoid pulling many different
    # static data versions, we will instead create a {champion_id -> champion_name} mapping and just access the champion's
    # ID from the match data (which it provides directly).
    champion_id_to_name_mapping = {champion.id: champion.name for champion in cass.get_champions(region=region)}
    played_champions = Counter()
    for match in match_history:
        champion_id = match.participants[summoner.name].champion.id
        champion_name = champion_id_to_name_mapping[champion_id]
        played_champions[champion_name] += 1
    print("Length of match history:", len(match_history))

    # Print the aggregated champion results
    print("Top 10 champions {} played:".format(summoner.name))
    for champion_name, count in played_champions.most_common(10):
        print(champion_name, count)
    print()

    match = match_history[0]
    print('Match ID:', match.id)

    p = match.participants[summoner]
    print("\nSince the match was created from a matchref, we only know one participant:")
    #print(p.summoner.name, 'playing', p.champion.name)
    print(p.id, p.summoner.region,  p.summoner.name, p.summoner.id, p.champion.id)

    print("\nNow pull the full match data by iterating over all the participants:")
    for p in match.participants:
        #print(p.summoner.name, 'playing', p.champion.name)
        print(p.id, p.summoner.region, p.summoner.name, p.summoner.id, p.champion.id, p.team.first_dragon)

    print("\nIterate over all the participants again and note that the data is not repulled:")
    for p in match.participants:
        #print(p.summoner.name, 'playing', p.champion.name)
        print(p.id, p.summoner.region,  p.summoner.name, p.summoner.id, p.champion.id, p.team.first_dragon)

    print("\nBlue team won?", match.blue_team.win)
    print("Red team won?", match.red_team.win)
    print("Participants on blue team:")
    for p in match.blue_team.participants:
        print(p.summoner.name)


if __name__ == "__main__":
    print_newest_match(name="머스켓", region="KR")

In [15]:
cass.get_match(id=3475505515).participants[0].summoner.name

'벤젠덴겐'

In [22]:
cass.get_match(id=3475505519).region

<Region.korea: 'KR'>

In [17]:
# match_id 3473000000~3475000000

In [18]:
cass.get_match(id=3473500000, region='KR').teams#.season

Making call: https://kr.api.riotgames.com/lol/match/v3/matches/3473500000


NotFoundError: No source returned a query result!